# Building a Desktop Database Application

### Generate an Sqlite database administration interface using Python's Tkinter module.

**Part of [Building a Desktop Database Application](./Building a Desktop Database Application.ipynb)**

**Developer: [David Schenck](https://github.com/zero2cx/)**

**App 5 - [The Python Mega Course](https://www.udemy.com/the-python-mega-course/) (Course Creator & Facilitator: [Ardit Sulce](http://pythonhow.com/author))**

Note: Python 3.6 or higher is required to execute this notebook application.

###### >> [Skip down to the application launcher](#launcher) <<

In [8]:
%reload_ext autoreload
%autoreload 2

In [9]:
import os, sys
import tkinter as tk

Import the notebooks that contain the **Database** class and the **seed_database()** function.

In [10]:
%%capture
%run db.ipynb
%run seed.ipynb

# HTML browser interface

### Function definition: main()

### Function definition: on_button_clicked()

The **main()** function creates the database and seeds it with data, when necessary. Then the function  generates and displays a Tk.Window user interface using the **Window** class of this notebook. Function **main()** is called by the **on_button_clicked()** event handler of the HTML buttons located below.

Note: Each seed file's first line determines the comma-separated column names. Each following line will form one comma-separated data record.

In [11]:
from ipywidgets import widgets
from IPython.display import HTML, clear_output

def main(db_name, title='Database Control Interface', seed=False):
    '''
    Generate the Tk window interface, and populate its form elements with data.
    '''
    if seed:
        seed_database(path='./data', name=db_name)
    ui = tk.Tk()
    Window(window=ui, title=title, db=Database(path='./data', name=db_name))
    ui.mainloop()

def on_button_clicked(button):
    '''
    Handle a button click event by calling function main().
    '''
    # Re-render the HTML form that's located below this cell.
    clear_output()
    display(HTML(description))
    display(chk_seed, box_buttons)
    display(HTML(line))
    
    # Inpsect the attributes of the form elements, then call function main().
    db_name = button.description
    seed = chk_seed.value
    chk_seed.value = False
    main(db_name=db_name, title=db_name, seed=seed)

# These HTML form elements will configure the launch of the Tk Window user interface.
line = '<hr style="height:4px;">'
description = f'''
<a name="launcher" style="color: black; text-decoration: none;">{line}</a>
<h1>Launch the application</h1>
<br>
<p>The buttons below will each generate the Tk Window user interface to administer a
different Sqlite database. When `Seed with fresh data` is checked, new table records
are generated from the seed data. <em>TODO: let the user upload or enter their own
custom seed-data.</em></p>
'''
chk_seed = widgets.Checkbox(value=False,description='Seed with fresh data', disabled=False)
button1 = widgets.Button(description='books')
button1.on_click(on_button_clicked)
button2 = widgets.Button(description='garden')
button2.on_click(on_button_clicked)
button3 = widgets.Button(description='ledzep')
button3.on_click(on_button_clicked)
box_buttons = widgets.HBox([button1, button2, button3])

# Render the form.
display(HTML(description))
display(chk_seed, box_buttons)
display(HTML(line))

Checkbox(value=False, description='Seed with fresh data')

HBox(children=(Button(description='books', style=ButtonStyle()), Button(description='garden', style=ButtonStyle()), Button(description='ledzep', style=ButtonStyle())))

# Application support classes

### Class definition: Window class

Generate and display a Tk window (GUI) using the **UserInterface** class of this notebook. The windowed interface administers the records in an Sqlite database table using the imported **Database** class of the [db.ipynb](./db.ipynb) notebook. Seed data is generated using the imported **seed_database()** function of the [seed.ipynb](./seed.ipynb) notebook.

In [12]:
class Window():

    def __init__(self, window, title, db):
        
        self.window = window
        self.window.wm_title(string=title)
        self.db = db
        
        # Build a list of fields (Tk.Label+Tk.Entry combo) from the table column-names.
        fields = []
        for column in self.db.get_column_names():
            fields.append(column)
        
        # Define a list of button names, each with an associated click-event handler.
        buttons = [
            ('view_all', self.view_collection),
            ('search', self.search_collection),
            ('add_new', self.add_item),
            ('update', self.update_item),
            ('delete', self.delete_item),
            ('close', self.window.destroy),
        ]
        
        # Initialize the user interface.
        self.ui = UserInterface(self.window, fields, buttons)
        self.view_collection()

    def view_collection(self):
        '''
        Display all records in the database table.
        '''
        self.ui.lst.delete(0, tk.END)
        for record in self.db.get_all_records():
            self.ui.lst.insert(tk.END, record)

    def search_collection(self):
        '''
        Search for all records that conform to the search criteria.
        '''
        val = {}
        for name in self.db.get_column_names():
            val[name] = self.ui.ent[name].get()
        records = self.db.get_records(**val)
        self.ui.lst.delete(0, tk.END)
        for record in records:
            self.ui.lst.insert(tk.END, (record))

    def add_item(self):
        '''
        Add a record to the database table.
        '''
        record = []
        for name in self.db.get_column_names():
            field = self.ui.ent[name].get()
            if not field:
                return
            record.append(field)
        self.db.add_record(record=record)
        self.view_collection()

    def update_item(self):
        '''
        Update a record in the database table.
        '''
        try:
            id = self.ui.selected[0]
        except NameError:
            return
        val = {}
        for name in self.db.get_column_names():
            val[name] = self.ui.ent[name].get()
            if not val[name]:
                return
        self.db.update_record(id=id, **val)
        self.view_collection()

    def delete_item(self):
        '''
        Delete a record from the database table.
        '''
        try:
            id = self.ui.selected[0]
        except NameError:
            return
        self.db.delete_record(id)
        self.view_collection()

### Class definition: UserInterface class

Generate Tk form elements according to the caller's specification and populate the caller's window attribute with them.

These form elements consists of:

- A flexible number of Tk.Entry elements, each associated with a Tk.Label element
- One Tk.Listbox element with an associated Tk.Scrollbar element
- Six Tk.Button elements

The form elements are placed into the Tk Window attribute of the caller, which is the **Window** class of this notebook.

In [13]:
class UserInterface():

    def __init__(self, window, fields, buttons):
        
        self.fields = fields
        self.window = window
        self.lbl = {}
        self.ent = {}
        self.btn = {}
        self.lst = None
        self.selected = None
        self.col = 0
        self.row = 0
        
        # Add all of the Tk form elements to the Tk.Window user interface.
        for field in fields:
            self.add_field(name=field, field_break=2)
        self.add_listbox()
        for i in range(len(buttons)):
            self.add_button(name=buttons[i][0], command=buttons[i][1])

    def add_field(self, name, field_break=2):
        '''
        Add an Entry and Label combo to the Window.
        '''
        # Add a Tk.Label element for this field.
        text = '%s: ' % (name.replace('_', ' ').title())
        label = tk.Label(master=self.window, text=text, height=2, width=12, anchor=tk.E)
        label.grid(row=self.row, column=self.col)
        self.col += 1
        
        # Add a Tk.Entry element for this field.
        entry = tk.Entry(master=self.window, textvariable=tk.StringVar(), width=16)
        entry.grid(row=self.row, column=self.col)
        self.col += 1
        
        # When field_break has been reached, set the next grid placement to land in column 0 of the next row.
        if self.col > (field_break - 1) * 2:
            self.col = 0
            self.row += 1
            
        self.lbl[name] = label
        self.ent[name] = entry

    def add_listbox(self):
        '''
        Add a Listbox and Scrollbar to the Window.
        '''
        self.row += 1
        self.col = 0
        
        # Add the Tk.Listbox element.
        listbox = tk.Listbox(master=self.window, width=28)
        listbox.grid(row=self.row, column=self.col, rowspan=6, columnspan=2, sticky=tk.E)
        self.col += 2
        
        # Add the Tk.Scrollbar element.
        scrollbar = tk.Scrollbar(master=self.window)
        scrollbar.grid(row=self.row, column=self.col, rowspan=6, sticky=tk.W)
        
        # Link the listbox with the scrollbar.
        listbox.configure(yscrollcommand=scrollbar.set)
        scrollbar.configure(command=listbox.yview)

        # Bind the listbox to a click-event handler.
        listbox.bind(sequence='<<ListboxSelect>>', func=self.get_selected)
        
        self.col += 1
        self.lst = listbox

    def add_button(self, name, command):
        '''
        Add a Button element to the Window.
        '''
        name = name.replace('_', ' ').title()
        button = tk.Button(master=self.window, text=name, width='12', command=command)
        button.grid(row=self.row, column=self.col, sticky=tk.E)
        self.row += 1
        self.btn[name] = button

    def get_selected(self, event):
        '''
        Determine which element of the Listbox has been clicked, if any.
        '''
        try:
            i = 0
            index = self.lst.curselection()[i]
            self.selected = self.lst.get(index)
            
            # Populate the Entry elements with the selected Listbox member's data.
            for field in self.fields:
                i += 1
                self.ent[field].delete(0, tk.END)
                self.ent[field].insert(tk.END, self.selected[i])
                
        except IndexError:
            return

### Information about this notebook:

In [14]:
from IPython.display import FileLink

print('Additional notebooks in this application (click to open in a new tab):')
display(FileLink('db.ipynb'))
display(FileLink('seed.ipynb'))

print('Associated files that contain the seed data (click to download):')
display(FileLink('./data/books.csv'))
display(FileLink('./data/garden.csv'))
display(FileLink('./data/ledzep.csv'))

license = '''
This software is licensed under the Gnu GPLv3
(c) 2017 David Schenck https://github.com/zero2cx/

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.
'''

Additional notebooks in this application (click to open in a new tab):


/home/zero2cx/projects/the-python-mega-course/App5/db.ipynb

/home/zero2cx/projects/the-python-mega-course/App5/seed.ipynb

Associated files that contain the seed data (click to download):


/home/zero2cx/projects/the-python-mega-course/App5/data/books.csv

/home/zero2cx/projects/the-python-mega-course/App5/data/garden.csv

/home/zero2cx/projects/the-python-mega-course/App5/data/ledzep.csv